In [1]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [2]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os
from tools import wait_for_node_finished
from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code, load_node

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [3]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
code = get_or_create_code('alamode.alm_opt',
                          computer_local,
                          '/home/max/Documents/alamode/bin/alm')
code_alm_opt = code
code = get_or_create_code('alamode.anphon',
                          computer_local,
                          '/home/max/Documents/alamode/bin/anphon')
code_anphon = code

code_alm_opt, code_anphon


(<Code: Remote code 'alamode.alm_opt' on localhost, pk: 59276, uuid: f978b37c-4404-4f61-ba70-2a2d6b8ad201>,
 <Code: Remote code 'alamode.anphon' on localhost, pk: 59277, uuid: dc42525d-6e01-4b6c-8568-b3fb3b14f6b2>)

In [4]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


{'CWD': 'run29'}


'/home/max/Documents/aiida-test/alamode-aiida/example/run29'

In [5]:
from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [6]:
g_cwd = nodebank.load("cwd")
g_dispfile_folder = nodebank.load("dispfile_folder")
g_displace_result = nodebank.load("displace_result")
g_superstructure = nodebank.load("superstructure")


In [7]:
g_prefix = nodebank.load('prefix')
g_wd = nodebank.load(f"{g_prefix.value}_almprefix")  # subdirectory名
g_norder = nodebank.load(f"{g_prefix.value}_norder")

g_prefix,g_norder,g_wd

(<Str: uuid: 2d95abd8-df96-46cf-a41e-921191bdf045 (pk: 59120) value: harmonic>,
 <Int: uuid: ea668f08-633f-4858-9bb2-7bd1fbfe8d0a (pk: 59121) value: 1>,
 <Str: uuid: 515c8f86-755c-4a1a-a70f-4c994ab22bbd (pk: 59123) value: Ga2N2_1x1x1_harmonic>)

# alm opt 

In [8]:
g_alamodeextract_future = nodebank.load(f'{g_prefix.value}_extract')


In [9]:
g_alamodeextract_future.outputs.dfset_file


<SinglefileData: uuid: bede38b1-4247-41f8-bd11-608fd3a9bff9 (pk: 59275)>

In [10]:
g_DFSET_file = g_alamodeextract_future.outputs.dfset_file
g_DFSET_file


<SinglefileData: uuid: bede38b1-4247-41f8-bd11-608fd3a9bff9 (pk: 59275)>

In [11]:
g_superstructure

<StructureData: uuid: 133fed56-6fda-4abf-b706-e09bb5ffbde2 (pk: 59115)>

In [12]:
code = code_alm_opt

builder = code.get_builder()
builder.structure = g_superstructure  # 　安定構造
builder.prefix = g_prefix
builder.cwd = g_cwd
builder.norder = g_norder
builder.dfset_file = g_DFSET_file

g_almoptfuture = submit(builder)
print(g_almoptfuture)
wait_for_node_finished(g_almoptfuture)


uuid: 547aa5dc-3924-4458-a614-37a3f606fa13 (pk: 59282) (aiida.calculations:alamode.alm_opt)
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
True True


In [13]:
nodebank.dump("anphonfile_xml", g_almoptfuture.outputs.input_ANPHON_file)


<SinglefileData: uuid: 65449d6d-0d4f-402a-b60f-ac8ee87375e4 (pk: 59285)>

In [14]:
g_almoptfuture.outputs.results.attributes

{'constraint': {'constraint_T-inv': 10,
  'num_free_HARMINC_FCs': 62,
  'constraint_R-inv_self': 0,
  'constraint_R-inv_cross': 0,
  'inqeuv_constraint_self': 108,
  'inqeuv_constraint_cross': 0},
 'outputfiles': {'input_ANPHON': 'harmonic.xml',
  'force_constants': 'harmonic.fcs'},
 'optimization': {'RSS': 0.00077648,
  'LMODEL': 'least-squares',
  'num_param': 170,
  'fitting_error': 1.9234,
  'num_free_param': 62,
  'rank_of_matrix': 62}}

# phonon dispersion

In [15]:
g_root_wd = nodebank.load("root_wd")
g_root_wd  # 実行root directory


<Str: uuid: ce1542b4-8c75-4a89-8ecc-eef6dac75026 (pk: 59093) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29>

In [16]:
g_wd  # subdirectory名


<Str: uuid: 515c8f86-755c-4a1a-a70f-4c994ab22bbd (pk: 59123) value: Ga2N2_1x1x1_harmonic>

In [17]:
g_property = Str(os.path.join(g_root_wd.value, "property")
                 )  # property directory
os.makedirs(g_property.value, exist_ok=True)
g_primstructure = nodebank.load("primstructure")  # primitive cell


In [18]:
code = code_anphon

builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_wd
builder.cwd = g_property
builder.fcsxml = g_almoptfuture.outputs.input_ANPHON_file
builder.mode = Str("phonons")
builder.phonons_mode = Str("band")

g_phbandfuture = submit(builder)
print(g_phbandfuture)
wait_for_node_finished(g_phbandfuture, 5)


uuid: f95ea744-c1d5-43df-abc8-17cdc9483c15 (pk: 59295) (aiida.calculations:alamode.anphon)
wait another 5 sec.
wait another 5 sec.
True True


In [19]:
g_primstructure.get_ase()


Atoms(symbols='Ga2N2', pbc=False, cell=[[3.19, 0.0, 0.0], [-1.595, 2.7626210380724, 0.0], [0.0, 0.0, 5.189]], masses=...)

In [20]:
g_diag = nodebank.load("diag")
print(g_diag.get_array('diag'))


[1 1 1]


In [21]:
# from alamode_aiida.alm_input import make_alm_uniform_kmesh
# g_qmesh_value = make_alm_uniform_kmesh(g_primstructure.get_ase(),kspacing=0.001)
g_qmesh = List(list=[10, 10, 10])  # qmeshの定義
g_qmesh.get_list()


[10, 10, 10]

In [22]:
g_almprefix = "_".join([g_primstructure.get_formula(),
                        "x".join(map(str, g_diag.get_array("diag"))),
                        g_prefix.value,
                       "q"+"x".join(map(str, g_qmesh.get_list()))
                        ])
print(g_almprefix)

g_wd_dos = Str(g_almprefix)
g_wd_dos  # dos用の識別子


Ga2N2_1x1x1_harmonic_q10x10x10


<Str: uuid: cb430894-4dcc-494b-b0fb-e334fc3830df (unstored) value: Ga2N2_1x1x1_harmonic_q10x10x10>

In [23]:
#codename = "anphon@tutor"
#code = Code.get_from_string(codename)
code = code_anphon


builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_wd_dos
builder.cwd = g_property
# builder.norder = g_norder
builder.fcsxml = g_almoptfuture.outputs.input_ANPHON_file
builder.mode = Str("phonons")
builder.phonons_mode = Str("dos")
builder.qmesh = g_qmesh

g_phdosfuture = submit(builder)
print(g_phdosfuture)
wait_for_node_finished(g_phdosfuture, 5)


uuid: bd09a773-6811-4ed5-9e74-077983495935 (pk: 59307) (aiida.calculations:alamode.anphon)
wait another 5 sec.
wait another 5 sec.
True True


# plot dispersion

In [24]:
g_wd


<Str: uuid: 515c8f86-755c-4a1a-a70f-4c994ab22bbd (pk: 59123) value: Ga2N2_1x1x1_harmonic>

In [25]:
phbandworkchain = WorkflowFactory('alamode.phband_img')
# g_img_filename = Str(f"{g_wd.value}_phband.pdf")
g_band_filenames = g_phbandfuture.outputs.phband_file
inputs = {"cwd": g_property, "band_filenames": g_band_filenames, "prefix": g_wd}
print(inputs)


{'cwd': <Str: uuid: 218ac17a-c77c-4830-b4bd-16300bbde52a (pk: 59288) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/property>, 'band_filenames': <SinglefileData: uuid: 8830bdc2-485b-4568-897b-776031be24a8 (pk: 59298)>, 'prefix': <Str: uuid: 515c8f86-755c-4a1a-a70f-4c994ab22bbd (pk: 59123) value: Ga2N2_1x1x1_harmonic>}


In [26]:

phbandimgworkchain = submit(phbandworkchain, **inputs)
print(phbandimgworkchain)
wait_for_node_finished(phbandimgworkchain)


uuid: 62d586fa-dd23-4902-af89-f6865ca42c2d (pk: 59315) (aiida.workflows:alamode.phband_img)
wait another 2 sec.
True True


In [27]:
phbandimgworkchain.outputs.img_file.filename  # g_property directoryの出力ファイル


'Ga2N2_1x1x1_harmonic_phband.pdf'

# plot DOS

In [28]:

phdosworkchain = WorkflowFactory('alamode.phdos_img')

inputs = {"cwd": g_property, "dos_filenames": g_phdosfuture.outputs.phdos_file,
          "prefix": g_wd_dos}
print(inputs)

g_dosfuture = submit(phdosworkchain, **inputs)
print(g_dosfuture)
wait_for_node_finished(g_dosfuture)
g_dosfuture.outputs.img_file.filename


{'cwd': <Str: uuid: 218ac17a-c77c-4830-b4bd-16300bbde52a (pk: 59288) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/property>, 'dos_filenames': <SinglefileData: uuid: 7c59400b-99ce-4d75-ad55-7f3f1683333d (pk: 59310)>, 'prefix': <Str: uuid: cb430894-4dcc-494b-b0fb-e334fc3830df (pk: 59300) value: Ga2N2_1x1x1_harmonic_q10x10x10>}
uuid: 3d45aa26-9446-41e5-9c3b-0266bc6ac1a1 (pk: 59320) (aiida.workflows:alamode.phdos_img)
wait another 2 sec.
True True


'Ga2N2_1x1x1_harmonic_q10x10x10_phdos.pdf'

## free energies

In [29]:
FreeenergyImgWorkChain = WorkflowFactory("alamode.freeenergy_img")
g_img_filename = Str("phdos_freeenergy.pdf")
g_inputs = {'cwd': g_property, 'prefix': g_wd_dos,
            'thermo_file': g_phdosfuture.outputs.thermo_file}

freeenergyimgfuture = nodebank.load("freeenergyimg", g_force)
if freeenergyimgfuture is None:
    # if True:
    freeenergyimgfuture = submit(FreeenergyImgWorkChain, **g_inputs)
    print(freeenergyimgfuture)
    wait_for_node_finished(freeenergyimgfuture)
    if freeenergyimgfuture.is_finished_ok:
        nodebank.dump("freeenergyimg", freeenergyimgfuture)


uuid: 50f9d916-151d-41fa-b429-955d1ab34590 (pk: 59325) (aiida.workflows:alamode.freeenergy_img)
wait another 2 sec.
True True


In [30]:
freeenergyimgfuture.outputs.img_file.list_object_names()


['Ga2N2_1x1x1_harmonic_q10x10x10_phfreeenergy.pdf']